In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📉 Convergence Hypothesis: Do Poor Countries Catch Up?

A fundamental question in economic growth is whether poorer countries tend to grow faster than richer countries, leading to a narrowing of income gaps over time. This idea is known as the **convergence hypothesis**.

The standard **Solow growth model**, due to diminishing returns to capital, predicts **conditional convergence**: countries with similar underlying characteristics (like saving rates, population growth rates, and technology levels - i.e., converging to the same steady state) should exhibit convergence. Poorer countries (further below their steady state) will grow faster than richer countries (closer to their steady state).

Testing **unconditional convergence** (simply looking if initially poor countries grow faster than initially rich ones, without controlling for other factors) often yields mixed results globally. However, it's a useful starting point to examine the predictions of the basic model.

This simulation generates artificial data for a set of countries to visually test for unconditional convergence by plotting average growth rates against initial income levels.

# ⚙️ Simulation Setup

We simulate data for $N$ countries over $T$ years:

1.  **Initial Income ($Y_0$):** Each country $i$ starts with a randomly assigned initial log income per capita, $\log(Y_{i0})$, drawn from a uniform distribution. This creates a range of initially rich and poor countries.
    $$ \log(Y_{i0}) \sim U(\text{min\_logY, max\_logY}) $$

2.  **Generating Growth Rates ($g_i$):** The core idea is to generate average annual growth rates ($g_i$) that depend *negatively* on the initial income level, plus some random noise, consistent with the convergence hypothesis.
    $$ g_i = \text{constant} + \beta \log(Y_{i0}) + \epsilon_i $$
    - $\beta$: The "true" convergence parameter. If $\beta < 0$, poorer countries (lower $\log(Y_{i0})$) tend to have higher growth $g_i$. This is the parameter we control with the `True Slope β` slider.
    - $\epsilon_i$: A random noise term ($\epsilon_i \sim N(0, \sigma^2_{\text{noise}})$) representing country-specific factors, shocks, or measurement error affecting growth, controlled by the `Noise Level` slider.
    * *(Note: The simulation code uses a simplified direct generation: `g = beta_true * log_Y0 + noise`)*

3.  **Final Income ($Y_T$):** Final income after $T$ years is calculated assuming growth was constant at the generated average rate $g_i$.
    $$ Y_{iT} = Y_{i0} e^{g_i T} $$

4.  **Realized Average Growth:** We calculate the *realized* average annual growth rate from the simulated $Y_0$ and $Y_T$.
    $$ g_{\text{avg}, i} = \frac{\log(Y_{iT}) - \log(Y_{i0})}{T} $$
    *(Note: Due to the noise term $\epsilon_i$, this $g_{\text{avg}, i}$ will be close to, but not exactly identical to, the $g_i$ generated in step 2).*

# 📊 Testing for Convergence: Regression Analysis

To test for convergence in our simulated data, we perform a simple linear regression:

$$g_{\text{avg}, i} = \text{Intercept} + \beta_{\text{estimated}} \log(Y_{i0}) + \text{error}_i$$

We plot the average growth rate ($g_{\text{avg}, i}$) against the log of initial income ($\log(Y_{i0})$) for all $N$ countries and fit a regression line.

**Interpretation:**

* **Estimated Slope ($\beta_{\text{estimated}}$):**
    * If $\beta_{\text{estimated}} < 0$ and is statistically significant (low p-value), it supports the unconditional convergence hypothesis within this simulated dataset. This means, on average, countries that started poorer grew faster.
    * If $\beta_{\text{estimated}} \ge 0$ or is not statistically significant, we find no evidence for unconditional convergence in the sample.
* **R-squared ($R^2$):** Measures the proportion of the variation in growth rates explained by the initial income level. A higher $R^2$ means initial income is a stronger predictor of growth in this sample.
* **Noise:** The `Noise Level` slider controls the variance of $\epsilon_i$. Higher noise makes the relationship between initial income and growth weaker, potentially obscuring convergence (lower $R^2$, higher p-value, $\beta_{\text{estimated}}$ closer to zero) even if the true $\beta$ is negative.

Experiment with the sliders (`True Slope β`, `Noise Level`, `Number of Countries`, `Years`) to see how they affect the visual pattern and the regression results.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider # Use IntSlider for N, T
from scipy.stats import linregress # For linear regression
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def convergence_simulation_plotter(N=30, T=20, growth_noise_std=0.01, beta_true=-0.02, log_y0_mean=8.0, log_y0_range=4.0):
    """
    Simulates cross-country data and tests the convergence hypothesis.

    Args:
        N (int): Number of countries to simulate.
        T (int): Time horizon in years.
        growth_noise_std (float): Standard deviation of the noise term added to growth rates.
        beta_true (float): The true underlying convergence parameter (slope).
        log_y0_mean (float): Mean for generating initial log GDP per capita.
        log_y0_range (float): Range (max-min) for generating initial log GDP per capita.
    """
    # Ensure integer inputs where needed
    N = int(N)
    T = int(T)
    if N < 5 or T < 1:
        print("Warning: N must be >= 5 and T >= 1.")
        return # Avoid running with invalid inputs

    # Set a seed for reproducibility
    np.random.seed(42)

    # Generate initial log GDP per capita from a uniform distribution
    log_y0_min = log_y0_mean - log_y0_range / 2
    log_y0_max = log_y0_mean + log_y0_range / 2
    log_Y0 = np.random.uniform(log_y0_min, log_y0_max, N)
    Y0 = np.exp(log_Y0) # Initial GDP levels

    # Generate underlying average growth rates based on convergence hypothesis + noise
    # g_i = constant + beta * log(Y0_i) + noise_i
    # We simplify by setting the constant implicitly such that mean growth is reasonable
    # A simple way is g_i = mean_growth + beta * (log(Y0_i) - mean(log(Y0))) + noise_i
    # Or even simpler as used here: g_i = some_base + beta*log(Y0_i) + noise
    # Let's center the deterministic part around 0 and add a base growth
    base_growth = 0.02 # Average growth around which deviations occur
    growth_deterministic = beta_true * (log_Y0 - log_y0_mean)
    growth_stochastic = np.random.normal(0, growth_noise_std, N)
    g_latent = base_growth + growth_deterministic + growth_stochastic

    # Calculate final income levels
    # YT = Y0 * exp(g*T)
    YT = Y0 * np.exp(g_latent * T)
    log_YT = np.log(YT)

    # Calculate realized average annual growth rate
    g_avg = (log_YT - log_Y0) / T

    # Perform linear regression: g_avg = intercept + slope * log_Y0
    try:
        # Use warnings context manager if scipy gives harmless warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            slope, intercept, r_value, p_value, std_err = linregress(log_Y0, g_avg)
        regression_successful = True
    except ValueError as e:
        print(f"Regression failed: {e}")
        slope, intercept, r_value, p_value, std_err = np.nan, np.nan, np.nan, np.nan, np.nan
        regression_successful = False

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(9, 6))

    # Scatter plot
    ax.scatter(log_Y0, g_avg * 100, s=60, color='darkorange', alpha=0.7, edgecolors='black', linewidth=0.5, label='Simulated Countries') # Plot growth in %

    # Regression line
    if regression_successful:
        x_line = np.array([log_Y0.min(), log_Y0.max()])
        y_line = (intercept + slope * x_line) * 100 # Convert growth to %
        ax.plot(x_line, y_line, color='black', linestyle='--', linewidth=2, label=f'Regression Line (β_est = {slope:.3f})')

    ax.set_title("Convergence Hypothesis Test (Simulated Data)")
    ax.set_xlabel("Log Initial GDP per capita (log Y₀)")
    ax.set_ylabel("Average Annual Growth Rate (g %)")
    ax.grid(True, linestyle='--', alpha=0.6)
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))

    ax.legend()
    plt.tight_layout()
    plt.show()

    # --- Display Regression Results ---
    if regression_successful:
        r_squared = r_value**2
        # Interpretation based on results
        if slope < 0 and p_value < 0.05:
            conclusion = f"✅ Supports convergence (p={p_value:.3g}). Poorer countries tend to grow faster in this simulation."
        elif slope < 0 and p_value >= 0.05:
            conclusion = f"⚠️ Negative slope, but not statistically significant (p={p_value:.3g}). Weak/no evidence for convergence."
        else:
            conclusion = f"❌ No convergence evidence (Slope ≥ 0 or insignificant, p={p_value:.3g})."

        results_md = f"""
        ### 📈 Regression Results: `Growth ~ Log Initial GDP`

        * **Estimated Slope (β_est):** {slope:.4f} (Std. Err: {std_err:.4f})
        * **Intercept:** {intercept*100:.2f}%
        * **P-value (for slope):** {p_value:.4g}
        * **R-squared:** {r_squared:.3f}
        * **Conclusion:** {conclusion}
            * *(Note: True underlying slope β was set to {beta_true:.3f})*
        """
    else:
        results_md = "### 📈 Regression Results:\n\n* Regression failed. Unable to estimate parameters."

    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'} # Allow longer descriptions
interact(convergence_simulation_plotter,
         N=IntSlider(value=30, min=10, max=200, step=5, description='Number of Countries (N):', style=style),
         T=IntSlider(value=20, min=5, max=50, step=1, description='Years (T):', style=style),
         growth_noise_std=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.002, description='Growth Noise (Std Dev):', readout_format='.3f', style=style),
         beta_true=FloatSlider(value=-0.02, min=-0.05, max=0.01, step=0.002, description='True Slope (β):', readout_format='.3f', style=style),
         log_y0_mean=FloatSlider(value=8.0, min=6.0, max=10.0, step=0.1, description='Mean log(Y₀):', readout_format='.1f', style=style),
         log_y0_range=FloatSlider(value=4.0, min=1.0, max=6.0, step=0.2, description='Range log(Y₀):', readout_format='.1f', style=style)
        );

interactive(children=(IntSlider(value=30, description='Number of Countries (N):', max=200, min=10, step=5, sty…

# 🏁 Conclusion

This simulation illustrates the concept of economic convergence.

* When the **True Slope $\beta$** is negative, the underlying tendency is for initially poorer countries (low $\log Y_0$) to grow faster.
* The **regression analysis** attempts to recover this underlying relationship from the simulated data.
* **Noise** in growth rates can obscure the true relationship, making it harder to detect convergence statistically (i.e., leading to a higher p-value or an estimated slope closer to zero), especially with a small number of countries ($N$).
* Real-world tests of unconditional convergence often find weak or no evidence, suggesting that factors *other* than just the initial income level (differences in saving, education, institutions, etc., leading to different steady states) are crucial determinants of growth - motivating the concept of **conditional convergence**.

# 📘 Convergence Hypothesis

This model tests if poorer countries grow faster — a key prediction of the Solow model with diminishing returns.

We simulate:
\[
g = \frac{1}{T} \log\left(\frac{Y_T}{Y_0}\right)
\quad \text{vs.} \quad \log(Y_0)
\]

- A **negative slope** in the regression suggests **convergence**
- A flat or positive slope suggests **divergence**

**Sources**:  
- GrowthEcon [Ch. 6](https://growthecon.com/StudyGuide/convergence.html)  
- Charles Jones, *Macroeconomics*, Ch. 6